In [1]:
# Train Transformer Model for Phase Classification

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score

# Load and prepare dataset
df = pd.read_csv('../data/synthetic_logs.csv')
df = df[df['phase'].notna()].copy()
phase_list = df["phase"].unique().tolist()
phase_map = {label: i for i, label in enumerate(phase_list)}
df['labels'] = df['phase'].map(phase_map)

dataset = Dataset.from_pandas(df[['log_text', 'labels']])
dataset = dataset.train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["log_text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(phase_map))

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results_phase",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs_phase",
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("../model/delay_phase_classifier")

# Save phase label map
import json
with open("../model/phase_map.json", "w") as f:
    json.dump(phase_map, f, indent=2)

print("Training complete. Model saved to ../model/delay_phase_classifier")


Map:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/7h/h_4y8hfd275gtzx2v47xmcl00000gn/T/ipykernel_93042/1189862006.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,2.006448,0.000000,0.000000
2,No log,1.206338,0.900000,0.897143
3,No log,0.983871,1.000000,1.000000
4,No log,0.978593,0.900000,0.895556


Training complete. Model saved to ../model/delay_phase_classifier
